In [1]:
!pip install gradio transformers langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 452.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━

In [2]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from langdetect import detect
import re

#  tokenizer تحميل النموذج والـ
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# قائمة بأكواد اللغات والأسماء
language_options = {
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Arabic": "ar",
}

#  دالة لترجمة الجمل
def translate_sentence(sentence, tgt_lang):
    encoded_text = tokenizer(sentence, return_tensors="pt")
    generated_tokens = model.generate(encoded_text["input_ids"], forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translation

# دالة لترجمة المقال كاملاً
def translate_article(article, tgt_lang):
    paragraphs = re.split(r'([\r\n]+)', article)
    for i, p in enumerate(paragraphs):
        if len(p.strip()) == 0:
            continue
        paragraphs[i] = translate_paragraph(p, tgt_lang)
    return ''.join(paragraphs)

#  دالة لترجمة الفقرات
def translate_paragraph(paragraph, tgt_lang):
    sentences = []
    cursor = 0
    for i, c in enumerate(paragraph):
        if c == '.':
            sentences.append(paragraph[cursor:i + 1])
            cursor = i + 1
    if paragraph and paragraph[-1] != '.':
        sentences.append(paragraph[cursor:])

    # ترجمة الجمل المجمعة في الفقرة
    return ' '.join(translate_sentence(s, tgt_lang) for s in sentences)

# دالة الكشف عن اللغة والترجمة إلى لغات متعددة
def detect_and_translate_multiple(text, tgt_langs):
    detected_lang = detect(text)
    src_lang_code = detected_lang if detected_lang in language_options.values() else "en"
    tokenizer.src_lang = src_lang_code

    translations = {}
    for tgt_lang in tgt_langs:
        translations[tgt_lang] = translate_article(text, language_options[tgt_lang])

    return "\n\n".join([f"{lang}: {translations[lang]}" for lang in translations])

# دالة لقراءة محتوى الملف النصي
def translate_file_in_chunks(file, tgt_langs, chunk_size=2000):
    translations = []
    try:
        # قراءة الملف النصي
        with open(file.name, "r", encoding="utf-8") as f:
            content = f.read()

        # ترجمة المحتوى
        translation = detect_and_translate_multiple(content, tgt_langs)
        translations.append(translation)

        return "\n\n".join(translations)
    except Exception as e:
        return f"Error during file translation: {str(e)}"

# واجهة Gradio
with gr.Blocks() as interface:
    gr.Markdown("## Text Translation with Auto Language Detection (Multiple Outputs)")

    with gr.Row(): #Fixed indentation
        input_text = gr.Textbox(label="Input Text")
        file_input = gr.File(label="Upload Text File (.txt)")

    tgt_langs = gr.CheckboxGroup(list(language_options.keys()), label="Target Languages", value=["English"])

    # مكون الإخراج: نتيجة الترجمة
    output_text = gr.Textbox(label="Translated Text")

    # زر الترجمة للنص المُدخل يدويًا
    translate_button = gr.Button("Translate Text")

    # زر الترجمة للملف المُحمل
    translate_file_button = gr.Button("Translate File")

    translate_button.click(
        detect_and_translate_multiple,
        inputs=[input_text, tgt_langs],
        outputs=output_text
    )

    translate_file_button.click(
        translate_file_in_chunks,
        inputs=[file_input, tgt_langs],
        outputs=output_text
    )

# تشغيل الواجهة
interface.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://87c1243ceeba694ae9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
